In [1]:
import numpy as np
import random
import sys
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [2]:
def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [3]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(0,3))
        self.board.addPiece('Pit','-',(0,3))
        self.board.addPiece('Wall','W',(2,3))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = randPair(0,self.board.size) #Row, Column
        self.board.components['Goal'].pos = (3,0)
        self.board.components['Pit'].pos = (3,1)
        self.board.components['Wall'].pos = (3,2)
        
        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridStatic()


    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [4]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

In [5]:
"""
from2dto1d:
    Args: 
        pos(tuple):2d position of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(integer): 1d position of the objects(Player, Wall, Goal, Pit)in the gridworld
""" 
def from2dto1d(pos):
    if pos == '(0, 0)':
        return 0
    if pos == '(0, 1)':
        return 1
    if pos == '(0, 2)':
        return 2
    if pos == '(0, 3)':
        return 3
    if pos == '(1, 0)':
        return 4
    if pos == '(1, 1)':
        return 5
    if pos == '(1, 2)':
        return 6
    if pos == '(1, 3)':
        return 7
    if pos == '(2, 0)':
        return 8
    if pos == '(2, 1)':
        return 9
    if pos == '(2, 2)':
        return 10
    if pos == '(2, 3)':
        return 11
    if pos == '(3, 0)':
        return 12
    if pos == '(3, 1)':
        return 13
    if pos == '(3, 2)':
        return 14
    if pos == '(3, 3)':
        return 15

In [6]:
def from1dto2d(pos):
    if pos == 0:
        return (0, 0)
    if pos == 1:
        return (0, 1)
    if pos == 2:
        return (0, 2)
    if pos == 3:
        return (0, 3)
    if pos == 4:
        return (1, 0)
    if pos == 5:
        return (1, 1)
    if pos == 6:
        return (1, 2)
    if pos == 7:
        return (1, 3)
    if pos == 8:
        return (2, 0)
    if pos == 9:
        return (2, 1)
    if pos == 10:
        return (2, 2)
    if pos == 11:
        return (2, 3)
    if pos == 12:
        return (3, 0)
    if pos == 13:
        return (3, 1)
    if pos == 14:
        return (3, 2)
    if pos == 15:
        return (3, 3)

In [7]:
"""
from_num_to_one_hot_encode:
    Args: 
        num(int): number from 0 to 15 representing the state of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(tuple): encoded binary code with the size of 16-bit 
"""
def from_num_to_one_hot_encode(num):
    en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
    if num == 0:
        en0 = 1
    if num == 1:
        en1 = 1
    if num == 2:
        en2 = 1
    if num == 3:
        en3 = 1
    if num == 4:
        en4 = 1
    if num == 5:
        en5 = 1
    if num == 6:
        en6 = 1
    if num == 7:
        en7 = 1
    if num == 8:
        en8 = 1
    if num == 9:
        en9 = 1
    if num == 10:
        en10 = 1
    if num == 11:
        en11 = 1
    if num == 12:
        en12 = 1
    if num == 13:
        en13 = 1
    if num == 14:
        en14 = 1
    if num == 15:
        en15 = 1
    return en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15

In [8]:
df = pd.read_csv("Reward_Changes_2.csv")
df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Q1_Next,Reward_1_Next,Q2_Next,Reward_2_Next,Q3_Next,Reward_3_Next,Q4_Next,Reward_4_Next,Max_Q_Next,Reward_Max_Next,Bellman_Operator
0,0,0,2,1,0,-0.356023,0.50,-10.000000,0.00,2.647438,0.62,-10.000000,0.00,-10.000000,0.00,2.647438,0.62,3.002694
1,1,2,2,1,0,9.709291,0.75,-10.000000,0.00,7.422089,0.62,-10.000000,0.00,7.962252,0.62,7.962252,0.62,10.000000
2,2,3,2,1,0,7.962252,0.62,-10.000000,0.00,6.114790,0.62,9.709291,0.75,-10.000000,0.00,9.709291,0.75,9.488361
3,3,4,2,1,0,2.647438,0.62,-0.356023,0.50,2.080958,0.62,-10.000000,0.00,6.384776,0.62,6.384776,0.62,6.366299
4,4,5,2,1,0,6.384776,0.62,-10.000000,0.00,3.147437,0.62,2.647438,0.62,7.422089,0.62,7.422089,0.62,7.299880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,6.419645,0.62,4.150536,0.62,10.481168,0.62,-10.000000,0.00,6.489353,0.62,10.481168,0.62,10.000000
50396,50396,13,13,14,15,9.970391,0.75,6.489353,0.62,-10.000000,0.00,10.481168,0.62,-10.000000,0.00,10.481168,0.62,10.000000
50397,50397,9,13,14,15,6.489353,0.62,3.969529,0.62,9.970391,0.75,6.419645,0.62,6.954754,0.62,9.970391,0.75,9.723352
50398,50398,12,13,14,15,10.481168,0.62,6.419645,0.62,-10.000000,0.00,-10.000000,0.00,9.970391,0.75,9.970391,0.75,9.723352


In [9]:
no_dup_df = df
no_dup_df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Q1_Next,Reward_1_Next,Q2_Next,Reward_2_Next,Q3_Next,Reward_3_Next,Q4_Next,Reward_4_Next,Max_Q_Next,Reward_Max_Next,Bellman_Operator
0,0,0,2,1,0,-0.356023,0.50,-10.000000,0.00,2.647438,0.62,-10.000000,0.00,-10.000000,0.00,2.647438,0.62,3.002694
1,1,2,2,1,0,9.709291,0.75,-10.000000,0.00,7.422089,0.62,-10.000000,0.00,7.962252,0.62,7.962252,0.62,10.000000
2,2,3,2,1,0,7.962252,0.62,-10.000000,0.00,6.114790,0.62,9.709291,0.75,-10.000000,0.00,9.709291,0.75,9.488361
3,3,4,2,1,0,2.647438,0.62,-0.356023,0.50,2.080958,0.62,-10.000000,0.00,6.384776,0.62,6.384776,0.62,6.366299
4,4,5,2,1,0,6.384776,0.62,-10.000000,0.00,3.147437,0.62,2.647438,0.62,7.422089,0.62,7.422089,0.62,7.299880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,6.419645,0.62,4.150536,0.62,10.481168,0.62,-10.000000,0.00,6.489353,0.62,10.481168,0.62,10.000000
50396,50396,13,13,14,15,9.970391,0.75,6.489353,0.62,-10.000000,0.00,10.481168,0.62,-10.000000,0.00,10.481168,0.62,10.000000
50397,50397,9,13,14,15,6.489353,0.62,3.969529,0.62,9.970391,0.75,6.419645,0.62,6.954754,0.62,9.970391,0.75,9.723352
50398,50398,12,13,14,15,10.481168,0.62,6.419645,0.62,-10.000000,0.00,-10.000000,0.00,9.970391,0.75,9.970391,0.75,9.723352


## Bellman Opeartor 100%

In [31]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/10, random_state = 0)

In [32]:
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [33]:
mlp.fit(X.values,y.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [34]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [35]:
def mlp_test_model(model_b, experience, mode='static', display=True):
    global counter
    i = 0
    test_game = Gridworld(mode=mode)
    if display:
        print("Initial State:")
        print(test_game.display())
    status = 1
    j = 0
    while(status == 1): #A
        p_curr = from2dto1d((str(test_game.board.components['Player'].pos)))
        pi = from2dto1d((str(test_game.board.components['Pit'].pos)))
        g = from2dto1d((str(test_game.board.components['Goal'].pos)))
        w = from2dto1d((str(test_game.board.components['Wall'].pos)))

        q_value_next = []

        test_game.makeMove('u')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('d')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('d')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('u')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('l')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            test_game.makeMove('r')
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('r')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('l')
        else:
            q_value_next.append(-10)
        
#         print(q_value_next)
        
        
        action_ = np.argmax(q_value_next)
#         print(action_)
        action = action_set[action_]
        
        experience[counter][0] = p_curr
        experience[counter][1] = pi
        experience[counter][2] = g
        experience[counter][3] = w
        experience[counter][4] = action_
        
    
        if display:
            print('Move #: %s; Taking action: %s' % (i, action))
        test_game.makeMove(action)
        p_next = from2dto1d((str(test_game.board.components['Player'].pos)))
        experience[counter][5] = p_next
        
        j += 1
        if display:
            print(test_game.display())
        reward = test_game.reward()
        experience[counter][6] = reward
        if reward > -1:
            experience[counter][7] = True
        else:
            experience[counter][7] = False
        experience[counter][8] =  q_value_next[action_]
        experience[counter][9] = q_value_next[0]
        experience[counter][10] = q_value_next[1]
        experience[counter][11] = q_value_next[2]
        experience[counter][12] = q_value_next[3]
        if reward != -1:
            if reward > 0:
                status = 2
                if display:
                    print("Game won! Reward: %s" % (reward,))
            else:
                status = 0
                if display:
                    print("Game LOST. Reward: %s" % (reward,))
        counter += 1
        i += 1
        if (i > 15):
            if display:
                print("Game lost; too many moves.")
            break
    
    win = True if status == 2 else False
    return win

In [36]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 880
0.88
0.88
Games played: 1000, # of wins: 869
0.869
1.749
Games played: 1000, # of wins: 881
0.881
2.63
Games played: 1000, # of wins: 892
0.892
3.522
Games played: 1000, # of wins: 894
0.894
4.4159999999999995
Games played: 1000, # of wins: 884
0.884
5.3
Games played: 1000, # of wins: 890
0.89
6.1899999999999995
Games played: 1000, # of wins: 864
0.864
7.053999999999999
Games played: 1000, # of wins: 886
0.886
7.9399999999999995
Games played: 1000, # of wins: 888
0.888
8.828
Win percentage: 88.27999999999999%


## Q-Value 100%

In [37]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X.values,y.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [38]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [39]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 919
0.919
0.919
Games played: 1000, # of wins: 927
0.927
1.846
Games played: 1000, # of wins: 919
0.919
2.765
Games played: 1000, # of wins: 932
0.932
3.697
Games played: 1000, # of wins: 913
0.913
4.61
Games played: 1000, # of wins: 928
0.928
5.538
Games played: 1000, # of wins: 934
0.934
6.472
Games played: 1000, # of wins: 932
0.932
7.404000000000001
Games played: 1000, # of wins: 921
0.921
8.325000000000001
Games played: 1000, # of wins: 912
0.912
9.237000000000002
Win percentage: 92.37000000000002%


## Bellman Operator 90%

In [12]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/10, random_state = 0)

In [13]:
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [14]:
mlp.fit(X_train.values,y_train.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [15]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [18]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 880
0.88
0.88
Games played: 1000, # of wins: 866
0.866
1.746
Games played: 1000, # of wins: 855
0.855
2.601
Games played: 1000, # of wins: 864
0.864
3.465
Games played: 1000, # of wins: 866
0.866
4.3309999999999995
Games played: 1000, # of wins: 855
0.855
5.186
Games played: 1000, # of wins: 880
0.88
6.066
Games played: 1000, # of wins: 858
0.858
6.9239999999999995
Games played: 1000, # of wins: 844
0.844
7.768
Games played: 1000, # of wins: 869
0.869
8.637
Win percentage: 86.37%


## Q-Value 90%

In [20]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/10, random_state = 0)

In [21]:
mlp.fit(X_train.values,y_train.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [22]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [23]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 937
0.937
0.937
Games played: 1000, # of wins: 942
0.942
1.879
Games played: 1000, # of wins: 935
0.935
2.814
Games played: 1000, # of wins: 921
0.921
3.7350000000000003
Games played: 1000, # of wins: 926
0.926
4.6610000000000005
Games played: 1000, # of wins: 921
0.921
5.582000000000001
Games played: 1000, # of wins: 943
0.943
6.525
Games played: 1000, # of wins: 922
0.922
7.447
Games played: 1000, # of wins: 920
0.92
8.367
Games played: 1000, # of wins: 930
0.93
9.297
Win percentage: 92.97000000000001%


## Bellman Operator 80%

In [41]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 2/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [42]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [43]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 881
0.881
0.881
Games played: 1000, # of wins: 891
0.891
1.772
Games played: 1000, # of wins: 868
0.868
2.64
Games played: 1000, # of wins: 874
0.874
3.5140000000000002
Games played: 1000, # of wins: 883
0.883
4.397
Games played: 1000, # of wins: 904
0.904
5.301
Games played: 1000, # of wins: 869
0.869
6.17
Games played: 1000, # of wins: 890
0.89
7.06
Games played: 1000, # of wins: 884
0.884
7.944
Games played: 1000, # of wins: 887
0.887
8.831
Win percentage: 88.31%


## Q-Value 80%

In [44]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 2/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [45]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 940
0.94
0.94
Games played: 1000, # of wins: 942
0.942
1.882
Games played: 1000, # of wins: 935
0.935
2.817
Games played: 1000, # of wins: 924
0.924
3.741
Games played: 1000, # of wins: 948
0.948
4.689
Games played: 1000, # of wins: 913
0.913
5.602
Games played: 1000, # of wins: 920
0.92
6.522
Games played: 1000, # of wins: 946
0.946
7.468
Games played: 1000, # of wins: 928
0.928
8.396
Games played: 1000, # of wins: 931
0.931
9.327000000000002
Win percentage: 93.27000000000002%


## Bellman Operator 70%

In [47]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 3/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [46]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [48]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 862
0.862
0.862
Games played: 1000, # of wins: 876
0.876
1.738
Games played: 1000, # of wins: 859
0.859
2.597
Games played: 1000, # of wins: 880
0.88
3.477
Games played: 1000, # of wins: 855
0.855
4.332
Games played: 1000, # of wins: 880
0.88
5.212
Games played: 1000, # of wins: 874
0.874
6.085999999999999
Games played: 1000, # of wins: 865
0.865
6.951
Games played: 1000, # of wins: 850
0.85
7.800999999999999
Games played: 1000, # of wins: 869
0.869
8.67
Win percentage: 86.7%


## Q-Value 70%

In [49]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 3/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [50]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 918
0.918
0.918
Games played: 1000, # of wins: 926
0.926
1.844
Games played: 1000, # of wins: 906
0.906
2.75
Games played: 1000, # of wins: 919
0.919
3.669
Games played: 1000, # of wins: 905
0.905
4.574
Games played: 1000, # of wins: 910
0.91
5.484
Games played: 1000, # of wins: 907
0.907
6.391
Games played: 1000, # of wins: 918
0.918
7.309
Games played: 1000, # of wins: 916
0.916
8.225
Games played: 1000, # of wins: 917
0.917
9.142
Win percentage: 91.41999999999999%


## Bellman Operator 60%

In [51]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 4/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [52]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [53]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 870
0.87
0.87
Games played: 1000, # of wins: 876
0.876
1.746
Games played: 1000, # of wins: 877
0.877
2.623
Games played: 1000, # of wins: 857
0.857
3.4800000000000004
Games played: 1000, # of wins: 848
0.848
4.328
Games played: 1000, # of wins: 862
0.862
5.19
Games played: 1000, # of wins: 874
0.874
6.064
Games played: 1000, # of wins: 867
0.867
6.931
Games played: 1000, # of wins: 859
0.859
7.79
Games played: 1000, # of wins: 868
0.868
8.658
Win percentage: 86.57999999999998%


## Q-Value 60%

In [54]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 4/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [55]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 920
0.92
0.92
Games played: 1000, # of wins: 920
0.92
1.84
Games played: 1000, # of wins: 932
0.932
2.7720000000000002
Games played: 1000, # of wins: 944
0.944
3.716
Games played: 1000, # of wins: 929
0.929
4.6450000000000005
Games played: 1000, # of wins: 915
0.915
5.5600000000000005
Games played: 1000, # of wins: 942
0.942
6.502000000000001
Games played: 1000, # of wins: 929
0.929
7.431000000000001
Games played: 1000, # of wins: 944
0.944
8.375
Games played: 1000, # of wins: 944
0.944
9.318999999999999
Win percentage: 93.19%


## Bellman Operator 50%

In [56]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 5/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [57]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [58]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 849
0.849
0.849
Games played: 1000, # of wins: 858
0.858
1.7069999999999999
Games played: 1000, # of wins: 831
0.831
2.538
Games played: 1000, # of wins: 852
0.852
3.3899999999999997
Games played: 1000, # of wins: 837
0.837
4.226999999999999
Games played: 1000, # of wins: 825
0.825
5.052
Games played: 1000, # of wins: 850
0.85
5.901999999999999
Games played: 1000, # of wins: 829
0.829
6.730999999999999
Games played: 1000, # of wins: 832
0.832
7.562999999999999
Games played: 1000, # of wins: 844
0.844
8.406999999999998
Win percentage: 84.06999999999998%


## Q-Value 50%

In [59]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 5/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [60]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 916
0.916
0.916
Games played: 1000, # of wins: 916
0.916
1.832
Games played: 1000, # of wins: 915
0.915
2.747
Games played: 1000, # of wins: 902
0.902
3.649
Games played: 1000, # of wins: 898
0.898
4.547
Games played: 1000, # of wins: 902
0.902
5.449
Games played: 1000, # of wins: 938
0.938
6.387
Games played: 1000, # of wins: 917
0.917
7.303999999999999
Games played: 1000, # of wins: 921
0.921
8.225
Games played: 1000, # of wins: 907
0.907
9.132
Win percentage: 91.32000000000001%


## Bellman Opearator 40%

In [61]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 6/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [62]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [63]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 860
0.86
0.86
Games played: 1000, # of wins: 851
0.851
1.7109999999999999
Games played: 1000, # of wins: 841
0.841
2.5519999999999996
Games played: 1000, # of wins: 849
0.849
3.401
Games played: 1000, # of wins: 855
0.855
4.256
Games played: 1000, # of wins: 856
0.856
5.112
Games played: 1000, # of wins: 842
0.842
5.954
Games played: 1000, # of wins: 850
0.85
6.803999999999999
Games played: 1000, # of wins: 866
0.866
7.669999999999999
Games played: 1000, # of wins: 864
0.864
8.533999999999999
Win percentage: 85.33999999999999%


## Q-Value 40%

In [64]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 6/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [65]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 872
0.872
0.872
Games played: 1000, # of wins: 893
0.893
1.7650000000000001
Games played: 1000, # of wins: 889
0.889
2.654
Games played: 1000, # of wins: 894
0.894
3.548
Games played: 1000, # of wins: 881
0.881
4.429
Games played: 1000, # of wins: 887
0.887
5.316000000000001
Games played: 1000, # of wins: 881
0.881
6.197000000000001
Games played: 1000, # of wins: 872
0.872
7.069000000000001
Games played: 1000, # of wins: 882
0.882
7.9510000000000005
Games played: 1000, # of wins: 898
0.898
8.849
Win percentage: 88.49000000000001%


## Bellman Operator 30%

In [66]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 7/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [67]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [68]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 812
0.812
0.812
Games played: 1000, # of wins: 823
0.823
1.635
Games played: 1000, # of wins: 814
0.814
2.449
Games played: 1000, # of wins: 837
0.837
3.2859999999999996
Games played: 1000, # of wins: 806
0.806
4.092
Games played: 1000, # of wins: 834
0.834
4.925999999999999
Games played: 1000, # of wins: 825
0.825
5.7509999999999994
Games played: 1000, # of wins: 803
0.803
6.553999999999999
Games played: 1000, # of wins: 815
0.815
7.369
Games played: 1000, # of wins: 821
0.821
8.19
Win percentage: 81.89999999999999%


## Q-Value 30%

In [71]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 7/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [72]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 855
0.855
0.855
Games played: 1000, # of wins: 861
0.861
1.716
Games played: 1000, # of wins: 868
0.868
2.584
Games played: 1000, # of wins: 876
0.876
3.46
Games played: 1000, # of wins: 877
0.877
4.337
Games played: 1000, # of wins: 850
0.85
5.186999999999999
Games played: 1000, # of wins: 857
0.857
6.044
Games played: 1000, # of wins: 865
0.865
6.909
Games played: 1000, # of wins: 856
0.856
7.765
Games played: 1000, # of wins: 879
0.879
8.644
Win percentage: 86.44000000000001%


## Bellman Operator 20%

In [74]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 8/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [75]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [76]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 781
0.781
0.781
Games played: 1000, # of wins: 756
0.756
1.537
Games played: 1000, # of wins: 797
0.797
2.334
Games played: 1000, # of wins: 796
0.796
3.13
Games played: 1000, # of wins: 783
0.783
3.913
Games played: 1000, # of wins: 798
0.798
4.711
Games played: 1000, # of wins: 771
0.771
5.482
Games played: 1000, # of wins: 774
0.774
6.256
Games played: 1000, # of wins: 770
0.77
7.026
Games played: 1000, # of wins: 792
0.792
7.818
Win percentage: 78.17999999999999%


## Q-Value 20%

In [77]:
from sklearn.model_selection import train_test_split
X = no_dup_df[['Player','Pit', 'Goal', 'Wall']]
y = no_dup_df[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 8/10, random_state = 0)
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
mlp.fit(X_train.values,y_train.values.ravel())

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [78]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 831
0.831
0.831
Games played: 1000, # of wins: 825
0.825
1.656
Games played: 1000, # of wins: 840
0.84
2.496
Games played: 1000, # of wins: 819
0.819
3.315
Games played: 1000, # of wins: 832
0.832
4.147
Games played: 1000, # of wins: 825
0.825
4.972
Games played: 1000, # of wins: 846
0.846
5.8180000000000005
Games played: 1000, # of wins: 843
0.843
6.6610000000000005
Games played: 1000, # of wins: 841
0.841
7.502000000000001
Games played: 1000, # of wins: 853
0.853
8.355
Win percentage: 83.55%
